# 模型文件地址
    
    https://github.com/PaddlePaddle/models/tree/develop/PaddleCV/image_classification
    
    wget http://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_pretrained.tar
    tar -xvf ResNet50_pretrained.tar
    
# 初始化环境

    pip install opencv-python
    pip install paddlepaddle==1.5
    
    

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import cv2
import time
import sys
import math
import numpy as np
import argparse
import functools

import paddle
import paddle.fluid as fluid

#加载自定义文件
from sdk.resnet import ResNet50
from sdk.attack_pp import *


In [2]:
"""
use_gpu=False

x = fluid.layers.data(name="x", shape=[1] ,dtype='int32')
x.stop_gradient=False
y = x*3

g = fluid.backward.gradients(targets=y, inputs=[x])[0]

place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())

result = exe.run(fluid.default_main_program(),
                 fetch_list=[y.name],
                 feed={ 'x':np.array([1]).astype('int32') })
#result = result[0][0]

print(result)
"""

'\nuse_gpu=False\n\nx = fluid.layers.data(name="x", shape=[1] ,dtype=\'int32\')\nx.stop_gradient=False\ny = x*3\n\ng = fluid.backward.gradients(targets=y, inputs=[x])[0]\n\nplace = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()\nexe = fluid.Executor(place)\nexe.run(fluid.default_startup_program())\n\nresult = exe.run(fluid.default_main_program(),\n                 fetch_list=[y.name],\n                 feed={ \'x\':np.array([1]).astype(\'int32\') })\n#result = result[0][0]\n\nprint(result)\n'

# 加载模型

In [3]:
#http://www.paddlepaddle.org.cn/documentation/docs/zh/1.5/user_guides/howto/prepare_data/index_cn.html

fluid.default_startup_program().random_seed = 1
fluid.default_main_program().random_seed = 1

class_dim=1000
image_shape=[3,224,224]
model_name="resnet50"
use_gpu=False
pretrained_model="ResNet50_pretrained"
TOPK=3

input_layer = fluid.layers.data(name='image', shape=image_shape, dtype='float32')
#设置为可以计算梯度
input_layer.stop_gradient=False

# model definition
model = ResNet50()
out = model.net(input=input_layer, class_dim=class_dim)
out = fluid.layers.softmax(out)

#评估模式
eval_program = fluid.default_main_program().clone(for_test=True)

fetch_list = [out.name]

place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())

#记载模型参数
fluid.io.load_persistables(exe, pretrained_model)

# 预测原始图片

In [4]:
img_path = "mug227.png"

img=process_img(img_path)

print(img.shape)

result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':img })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]
print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()

(1, 3, 224, 224)
Test-score: [0.63326067 0.23805845 0.0749701 ], class [504 967 968]


In [5]:
#同样一张图片 训练模式和预测模式下的结果不一样

#评估模式
demo1_program = fluid.default_main_program().clone(for_test=True)

print("eval model")

for _ in range(2):
    result = exe.run(demo1_program,
                 fetch_list=fetch_list,
                 feed={ 'image':img })
    result = result[0][0]

    pred_label = np.argsort(result)[::-1][:TOPK]
    print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
    sys.stdout.flush()
    
print("train model")
       
#训练模式
demo2_program = fluid.default_main_program().clone(for_test=False)
demo2_program.random_seed = 1

for _ in range(2):
    result = exe.run(demo2_program,
                 fetch_list=fetch_list,
                 feed={ 'image':img })
    result = result[0][0]

    pred_label = np.argsort(result)[::-1][:TOPK]
    print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
    sys.stdout.flush()
    


                
#训练模式
demo3_program = fluid.default_main_program().clone(for_test=True)

for op in demo3_program.block(0).ops:
    #print("op type is {}".format(op.type))
    if op.type in ["batch_norm"]:
        # 兼容旧版本 paddle
        if hasattr(op, 'set_attr'):
            op.set_attr('is_test', False)
            op.set_attr('use_global_stats', True)
        else:
            op._set_attr('is_test', False)
            op._set_attr('use_global_stats', True)
            
            
            

print("debug model")

for _ in range(2):
    result = exe.run(demo3_program,
                 fetch_list=fetch_list,
                 feed={ 'image':img })
    result = result[0][0]

    pred_label = np.argsort(result)[::-1][:TOPK]
    print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
    sys.stdout.flush()

eval model
Test-score: [0.63326067 0.23805845 0.0749701 ], class [504 967 968]
Test-score: [0.63326067 0.23805845 0.0749701 ], class [504 967 968]
train model
Test-score: [0.00893321 0.00554745 0.00553076], class [463 899 600]
Test-score: [0.00893321 0.00554745 0.00553076], class [463 899 600]
debug model
Test-score: [0.5617399  0.2568845  0.07062895], class [504 967 968]
Test-score: [0.5617399  0.2568845  0.07062895], class [504 967 968]


# 调用FGSM算法 无定向攻击

In [6]:
img_path = "mug227.png"
img=process_img(img_path)

init_prog(fluid.default_main_program())

#生成对抗样本
adv=FGSM(o=img,input_layer=input_layer,output_layer=out,step_size=16.0/256.0,isTarget=False,target_label=0)

    
result = exe.run(eval_program,
                 fetch_list=fetch_list,
                 feed={ 'image':adv })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]

print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()

RuntimeError: boost::bad_get: failed value get using boost::get